In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '../Config')
from lablib import *

# Definizione funzioni

In [ ]:
#Legge per la figura di interferenza di Fabry-Perot
lambda_ = 632.8e-9 # lunghezza d'onda del laser

def FP_maximum(N, d, delta):
    cos = N*lambda_/(2*d) - (delta*lambda_/(d*np.pi*4)) # coseno dell'angolo di incidenza
    return -cos

#Legge per calibrazione del nonio in Fabry-Perot

def FP_calib(dN):
    d = dN*lambda_/2    #variazione di distanza tra le lastre 
    return d

def MC_nVetro(dN, d, theta):
    n = (2*d - dN*lambda_)*(1-np.cos(theta))/(2*d * (1-np.cos(theta)) - dN * lambda_)
    return n

# Fabry-Perot

## Specchio

Misure di FP1 prese in mm \
sensibilità calibro 0.05mm

In [ ]:
dati_fp1 = pd.read_excel('FP1.xlsx')
raggi_fp1 = dati_fp1[:].dropna().values.T / 1000 # raggi in m
d_sorgente = 1.375 # distanza da sorgente in m

N_fp1 = list(range(1, len(raggi_fp1[0])+1))
massimi_fp1 = [[np.cos(np.arctan(raggi_fp1[i][j])) for j in range(len(raggi_fp1[i]))] for i in range(len(raggi_fp1))]

yerr_temp = [.00005]*len(N_fp1)

In [ ]:
par1, val1, ers1, *other1 = LS_fit(N_fp1, massimi_fp1[0], yerr_temp, FP_maximum, d = 0.002, delta = 5e4, disp = 0)
par2, val2, ers2, *other2 = LS_fit(N_fp1, massimi_fp1[1], yerr_temp, FP_maximum, d = 0.002, delta = 5e4, disp = 0)
par3, val3, ers3, *other3 = LS_fit(N_fp1, massimi_fp1[2], yerr_temp, FP_maximum, d = 0.002, delta = 5e4, disp = 1)
par4, val4, ers4, *other4 = LS_fit(N_fp1, massimi_fp1[3], yerr_temp, FP_maximum, d = 0.002, delta = 5e4, disp = 0)

In [ ]:
for i in range(len(par1)):
    print(f'{par1[i]}1: {val1[i]:.3g} ± {ers1[i]:.3g}')
    print(f'{par2[i]}2: {val2[i]:.3g} ± {ers2[i]:.3g}')
    print(f'{par3[i]}3: {val3[i]:.3g} ± {ers3[i]:.3g}')
    print(f'{par4[i]}4: {val4[i]:.3g} ± {ers4[i]:.3g}')
    if i == 0:
        print("--------------------")

print("--------------------")

print(f"Chi 1: {(other1[2]/other1[1]):.3g} - Pvalue 1: {other1[0]:.3g}")
print(f"Chi 2: {(other2[2]/other2[1]):.3g} - Pvalue 2: {other2[0]:.3g}")
print(f"Chi 3: {(other3[2]/other3[1]):.3g} - Pvalue 3: {other3[0]:.3g}")
print(f"Chi 4: {(other4[2]/other4[1]):.3g} - Pvalue 4: {other4[0]:.3g}")

## Frange

20 micrometri -> passonomio \
N frange = 56, 59, 60, 60, 64, 56 \
Approssimiamo $\cos\theta$ a 1

In [ ]:
delta_N_fp = [56, 56, 59, 60, 60, 64] # se abbiamo tempo sarebbero da riprendere
delta_d_fp = [FP_calib(delta_N[i]) for i in range(len(delta_N))]
print(delta_d)

# Michelson

## Frange

In [ ]:
delta_N_mc = [60, 64]
delta_d_mc = [FP_calib(delta_N_mc[i]) for i in range(len(delta_N_mc))]
print(delta_d_mc)

## Indice rifrazione aria

## Indice rifrazione vetro

In [ ]:
thetai1, thetaf1, N1 = *np.deg2rad([1.0, 6.0]), 24
thetai2, thetaf2, N2 = *np.deg2rad([1.0, 5.0]), 16
thetai3, thetaf3, N3 = *np.deg2rad([1.0, 4.0]), 8

d_vetro = .005 # non ho idea quale sia in realtà
# ho messo questo perche' n del vetro e' circa 1.5 

n_vetro = [MC_nVetro(N1, d_vetro, thetai1-thetaf1), MC_nVetro(N2, d_vetro, thetai2-thetaf2), MC_nVetro(N3, d_vetro, thetai3-thetaf3)]

for i in range(len(n_vetro)):
    print(f'n_vetro_{i+1}: {n_vetro[i]:.3g}')
print(f"n_vetro medio: {np.mean(n_vetro):.3g} ± {np.std(n_vetro):.3g}")

## Righello